## Q1. Getting the embeddings model

In [41]:
import requests
from sentence_transformers import SentenceTransformer

# Fetching the documents
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

# Loading the model
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

# Creating the embedding for the user question
user_question = "I just discovered the course. Can I still join it?"
embedding = embedding_model.encode(user_question)

# Retrieve the first value of the resulting vector
first_value = embedding[0]
print(first_value)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
0.078222655


## Q2. Creating the embeddings

In [38]:
import requests
import numpy as np
from sentence_transformers import SentenceTransformer

# Fetching the documents
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

# Loading the model
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

# Create a list to hold the embeddings
embeddings = []

# Iterate over each document to create embeddings for question and answer
for doc in documents:
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

# Convert the list of embeddings to a numpy array
X = np.array(embeddings)

# Print the shape of the matrix X
print(X.shape)

(948, 768)


## Q3. Search

In [56]:
import requests
import numpy as np
from sentence_transformers import SentenceTransformer

# Fetching the documents
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

# Loading the model
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

# Create a list to hold the embeddings
embeddings = []

# Iterate over each document to create embeddings for question and answer
for doc in documents:
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

# Convert the list of embeddings to a numpy array
X = np.array(embeddings)

# Create the embedding for the user question
user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)

# Compute the cosine similarity scores
scores = X.dot(v)

# Find the highest score
highest_score = np.max(scores)
print(highest_score)

0.6506573


## Vector search

In [57]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'id': '7842b56a'},
 {'text': 'Star the r

## Q4. Hit-rate for our search engine

In [ ]:
# docker run -it \
#     --rm \
#     --name elasticsearch \
#     -p 9200:9200 \
#     -p 9300:9300 \
#     -e "discovery.type=single-node" \
#     -e "xpack.security.enabled=false" \
#     docker.elastic.co/elasticsearch/elasticsearch:8.4.3

In [43]:
# Initialize Elasticsearch
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': '7d2aec048f7e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'uKMEKEiqRCer0R_X0lFQ-w', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [44]:
import json
import requests
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

# Load documents
with open('eval/documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

# Initialize model
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

# Initialize Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

# Define index settings
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

# Recreate index
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [45]:
# Index documents
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = embedding_model.encode(question)
    doc['text_vector'] = embedding_model.encode(text)
    doc['question_text_vector'] = embedding_model.encode(qt)

100%|████████████████████████████████████████████████████████████████████████| 948/948 [03:26<00:00,  4.59it/s]


In [46]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████| 948/948 [00:19<00:00, 47.58it/s]


In [47]:
# Function for k-NN search
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [48]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = embedding_model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

In [49]:
df_ground_truth = pd.read_csv('eval/ground-truth-data.csv')
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [50]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [51]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [52]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [53]:
# Function for question vector search using VectorSearchEngine
def question_vector_search(q):
    question = q['question']
    v_q = embedding_model.encode(question)
    return search_engine.search(v_q, num_results=5)

In [58]:
# Evaluate the search engine
evaluation_results = evaluate(ground_truth, question_vector_search)
print(f"Hit rate: {evaluation_results['hit_rate']}")
print(f"MRR: {evaluation_results['mrr']}")

100%|██████████████████████████████████████████████████████████████████████| 1830/1830 [02:18<00:00, 13.20it/s]

Hit rate: 0.9218579234972678
MRR: 0.8202459016393446


## Q5. Indexing with Elasticsearch

In [59]:
# Perform a k-NN search using the same query from user_question
user_question = "I just discovered the course. Can I still join it?"
v_query = embedding_model.encode(user_question)

def elastic_search_knn(field, vector, course=None):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000
    }
    if course:
        knn["filter"] = {
            "term": {
                "course": course
            }
        }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

# Search for the user question
results = elastic_search_knn('question_vector', v_query)
highest_score_doc_id = results[0]['id']
print(f"Document ID with the highest score: {highest_score_doc_id}")

Document ID with the highest score: ee58a693


## Q6. Hit-rate for Elasticsearch

In [60]:
# Function for question vector search using Elasticsearch
def question_vector_search_elastic(q):
    question = q['question']
    course = q['course']
    v_q = embedding_model.encode(question)
    return elastic_search_knn('question_vector', v_q, course)

# Evaluate the Elasticsearch-based search engine
evaluation_results_elastic = evaluate(ground_truth, question_vector_search_elastic)
print(f"Hit rate for Elasticsearch: {evaluation_results_elastic['hit_rate']}")
print(f"MRR for Elasticsearch: {evaluation_results_elastic['mrr']}")

100%|██████████████████████████████████████████████████████████████████████| 1830/1830 [01:12<00:00, 25.28it/s]

Hit rate for Elasticsearch: 0.8076502732240437
MRR for Elasticsearch: 0.6985519125683067
